In [2]:
import torch
from matplotlib import pyplot as plt
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18
from tqdm.notebook import tqdm
import numpy as np
import torch.nn.functional as F
import os
import pickle
from attack_pgd import pgd
from torch.nn.parallel import DataParallel
import logging
from torch.utils.tensorboard import SummaryWriter
from torchcontrib.optim import SWA
# Set the device
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set the random seed for reproducibility
torch.manual_seed(42)

# Define constants
UNLABELED_BS = 50
TRAIN_BS = 50
TEST_BS = 128
num_train_samples = 5000
samples_per_class = int(num_train_samples / 10)

num_validation_samples = int(num_train_samples * 0.1)


# Define transformations for data augmentation
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = torch.utils.data.DataLoader(trainset, batch_size=TRAIN_BS, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(testset, batch_size=TEST_BS, shuffle=False, num_workers=2)

# Create labeled and unlabeled datasets
labeled_indices = []
unlabeled_indices = []

# Split the trainset into labeled and unlabeled data
for class_index in range(10):
    class_indices = torch.where(torch.Tensor(trainset.targets) == class_index)[0]
    labeled_indices.extend(class_indices[:samples_per_class])
    unlabeled_indices.extend(class_indices[samples_per_class:])

labeled_dataset = torch.utils.data.Subset(trainset, labeled_indices)
unlabeled_dataset = torch.utils.data.Subset(trainset, unlabeled_indices)

# Create data loaders for labeled and unlabeled data
labeled_loader = torch.utils.data.DataLoader(labeled_dataset, batch_size=TRAIN_BS, shuffle=True, num_workers=2)
unlabeled_loader = torch.utils.data.DataLoader(unlabeled_dataset, batch_size=UNLABELED_BS, shuffle=True, num_workers=2)

#labeled_train_subset, labeled_validation_subset = torch.utils.data.random_split(
#    labeled_dataset, [num_train_samples - num_validation_samples, num_validation_samples])

#labeled_loader = torch.utils.data.DataLoader(
#    labeled_train_subset, batch_size=TRAIN_BS, shuffle=True, num_workers=2)

#labeled_validation_loader = torch.utils.data.DataLoader(
#    labeled_validation_subset, batch_size=TEST_BS, shuffle=False, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


In [3]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.nn import functional as F
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
from torch.nn.utils import weight_norm

import sys
import math
import itertools
from torch.autograd import Variable, Function

In [4]:
# For cifar_cnn
class CNN(nn.Module):
    """
    CNN from Mean Teacher paper
    """

    def __init__(self, num_classes=10, isL2 = False, dropRatio = 0.0):
        super(CNN, self).__init__()

        self.isL2 = isL2

        self.activation = nn.LeakyReLU(0.1)
        self.conv1a = weight_norm(nn.Conv2d(3, 128, 3, padding=1))
        self.bn1a = nn.BatchNorm2d(128)
        self.conv1b = weight_norm(nn.Conv2d(128, 128, 3, padding=1))
        self.bn1b = nn.BatchNorm2d(128)
        self.conv1c = weight_norm(nn.Conv2d(128, 128, 3, padding=1))
        self.bn1c = nn.BatchNorm2d(128)
        self.mp1 = nn.MaxPool2d(2, stride=2, padding=0)
        # self.drop1  = nn.Dropout(0.5)
        # self.drop1  = nn.Dropout(dropRatio)
        self.drop  = nn.Dropout(dropRatio)

        self.conv2a = weight_norm(nn.Conv2d(128, 256, 3, padding=1))
        self.bn2a = nn.BatchNorm2d(256)
        self.conv2b = weight_norm(nn.Conv2d(256, 256, 3, padding=1))
        self.bn2b = nn.BatchNorm2d(256)
        self.conv2c = weight_norm(nn.Conv2d(256, 256, 3, padding=1))
        self.bn2c = nn.BatchNorm2d(256)
        self.mp2 = nn.MaxPool2d(2, stride=2, padding=0)
        # self.drop2  = nn.Dropout(0.5)
        # self.drop2  = nn.Dropout(dropRatio)

        self.conv3a = weight_norm(nn.Conv2d(256, 512, 3, padding=0))
        self.bn3a = nn.BatchNorm2d(512)
        self.conv3b = weight_norm(nn.Conv2d(512, 256, 1, padding=0))
        self.bn3b = nn.BatchNorm2d(256)
        self.conv3c = weight_norm(nn.Conv2d(256, 128, 1, padding=0))
        self.bn3c = nn.BatchNorm2d(128)
        self.ap3 = nn.AvgPool2d(6, stride=2, padding=0)

        self.fc1 =  weight_norm(nn.Linear(128, num_classes))
        self.fc2 =  weight_norm(nn.Linear(128, num_classes))

    def forward(self, x, debug=False):
        x = self.activation(self.bn1a(self.conv1a(x)))
        x = self.activation(self.bn1b(self.conv1b(x)))
        x = self.activation(self.bn1c(self.conv1c(x)))
        x = self.mp1(x)
        x = self.drop(x)

        x = self.activation(self.bn2a(self.conv2a(x)))
        x = self.activation(self.bn2b(self.conv2b(x)))
        x = self.activation(self.bn2c(self.conv2c(x)))
        x = self.mp2(x)
        x = self.drop(x)

        x = self.activation(self.bn3a(self.conv3a(x)))
        x = self.activation(self.bn3b(self.conv3b(x)))
        x = self.activation(self.bn3c(self.conv3c(x)))
        x = self.ap3(x)

        x = x.view(-1, 128)
        if self.isL2:
            x = F.normalize(x)
        # return self.fc1(x), self.fc2(x), x
        return self.fc1(x)#, self.fc2(x), x

        

In [ ]:
"""
from models.wideresnet import WideResNet
import torch
from torch.nn import Sequential, Module

def get_model(name, num_classes=10, normalize_input=False):
    name_parts = name.split('-')
    if name_parts[0] == 'wrn':
        depth = int(name_parts[1])
        widen = int(name_parts[2])
        model = WideResNet(
            depth=depth, num_classes=num_classes, widen_factor=widen)
    if normalize_input:
        model = Sequential(NormalizeInput(), model)

    return model

class NormalizeInput(Module):
    def __init__(self, mean=(0.4914, 0.4822, 0.4465),
                 std=(0.2023, 0.1994, 0.2010)):
        super().__init__()

        self.register_buffer('mean', torch.Tensor(mean).reshape(1, -1, 1, 1))
        self.register_buffer('std', torch.Tensor(std).reshape(1, -1, 1, 1))

    def forward(self, x):
        return (x - self.mean) / self.std
"""

In [5]:
from __future__ import print_function
import os
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
from torchvision import datasets, transforms

#from models.wideresnet import *
#from models.resnet import *
#from losses import trades_loss

In [6]:
model_dir = './rst_adv/combined_loader/13CNN/mixup'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(message)s",
    handlers=[
        logging.FileHandler(os.path.join(model_dir, 'fly_soft_SWA70_CNN_lowBS_50_50_50k_training.log')),
        logging.StreamHandler()
    ])
logger = logging.getLogger()

In [7]:
##############################################################################

def loss_soft_reg_ep(preds, labels, soft_labels, device):
    num_classes = 10
    prob = F.softmax(preds, dim=1)
    prob_avg = torch.mean(prob, dim=0)
    p = torch.ones(num_classes).to(device) / args.num_classes

    L_c = -torch.mean(torch.sum(soft_labels * F.log_softmax(preds, dim=1), dim=1))   # Soft labels
    L_p = -torch.sum(torch.log(prob_avg) * p)
    L_e = -torch.mean(torch.sum(prob * F.log_softmax(preds, dim=1), dim=1))

    loss = L_c + 0.8 * L_p + 0.4 * L_e
    return prob, loss

##############################################################################
def mixup_data(x, y, alpha=1.0, device='cuda'):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    if device=='cuda':
        index = torch.randperm(batch_size).to(device)
    else:
        index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam





In [8]:
"""
def loss_mixup_reg_ep(preds, labels, targets_a, targets_b, device, lam):
    num_classes = 10
    prob = F.softmax(preds, dim=1)
    prob_avg = torch.mean(prob, dim=0)
    p = torch.ones(num_classes).to(device) / num_classes

    mixup_loss_a = -torch.mean(torch.sum(targets_a * F.log_softmax(preds, dim=1), dim=1))
    mixup_loss_b = -torch.mean(torch.sum(targets_b * F.log_softmax(preds, dim=1), dim=1))
    mixup_loss = lam * mixup_loss_a + (1 - lam) * mixup_loss_b

    L_p = -torch.sum(torch.log(prob_avg) * p)
    L_e = -torch.mean(torch.sum(prob * F.log_softmax(preds, dim=1), dim=1))

    loss = mixup_loss + 0.8 * L_p + 0.4 * L_e
    return prob, loss
"""

In [12]:

def loss_mixup_reg_ep(preds, labels, targets_a, targets_b, device, lam):
    num_classes = preds.size(1)  # Get the number of classes from preds
    prob = F.softmax(preds, dim=1)
    prob_avg = torch.mean(prob, dim=0)
    p = torch.ones(num_classes).to(device) / num_classes

    # Ensure that targets_a and targets_b have the correct shape
    targets_a_onehot = F.one_hot(targets_a, num_classes=num_classes).float()
    targets_b_onehot = F.one_hot(targets_b, num_classes=num_classes).float()

    mixup_loss_a = -torch.mean(torch.sum(targets_a_onehot * F.log_softmax(preds, dim=1), dim=1))
    mixup_loss_b = -torch.mean(torch.sum(targets_b_onehot * F.log_softmax(preds, dim=1), dim=1))
    mixup_loss = lam * mixup_loss_a + (1 - lam) * mixup_loss_b

    L_p = -torch.sum(torch.log(prob_avg) * p)
    L_e = -torch.mean(torch.sum(prob * F.log_softmax(preds, dim=1), dim=1))

    loss = mixup_loss + 0.8 * L_p + 0.4 * L_e
    return prob, loss


In [13]:

step_size= 0.007
beta = 6
num_steps = 10
epsilon=0.031
#lr = 0.1
batch_size= 128
test_batch_size= 128
epochs = 150
weight_decay= 2e-4
momentum = 0.9
seed = 1
log_interval= 100
#model_dir= './rst_adv/trades'
distance = 'l_inf'
eval_attack_batches = 1
pgd_num_steps = 10
pgd_step_size = 0.007

swa_start = 70
swa_freq = 5


def train_new(model, device, labeled_loader, unlabeled_loader, optimizer, swa_optimizer, epoch):    

    #loss_ema = 0.
    alpha=1
    labeled_iter = iter(labeled_loader)    
    num_iter = (len(unlabeled_loader.dataset)//TRAIN_BS)+1
    for batch_idx, (x_unlabeled, _) in enumerate(unlabeled_loader):      
        try:
            x_labeled, y_labeled = next(labeled_iter)
        except StopIteration:
            labeled_iter = iter(labeled_loader)
            x_labeled, y_labeled = next(labeled_iter) 
        model.eval()
        with torch.no_grad():
            x_unlabeled = x_unlabeled.to(device)
            #pseudo_labels = model(x_unlabeled.to(device)).argmax(dim=1)
            #logits_unlabeled = model(x_unlabeled.to(device))
            # Softmax normalization to obtain probabilities for each class
            #pseudo_labels = F.softmax(logits_unlabeled, dim=1)
            
            logits_pseudo_labels = model(x_unlabeled.to(device))
            pseudo_labels = F.softmax(logits_pseudo_labels, dim=1)
            
        
        model.train()
        
        # concatenate (x_labeled, y_labeled) and (x_unlabeled, pseudo_labels)
        x_labeled = x_labeled.to(device)
        y_labeled = y_labeled.to(device)
        #print(y_labeled.shape)
        #print(y_labeled[0])
        # Add a new dimension to y_labeled to make it compatible for concatenation
        num_classes = 10
        y_labeled_onehot = F.one_hot(y_labeled, num_classes).float()
        #print(y_labeled_onehot.shape)
        #print(y_labeled_onehot[0])
        pseudo_labels = pseudo_labels.to(device)
        y_labeled_onehot = y_labeled_onehot.to(device)
        #print(pseudo_labels.shape)
        
        data = torch.cat([x_labeled,x_unlabeled],dim=0)
        target = torch.cat([y_labeled_onehot,pseudo_labels],dim=0)
        #target = torch.cat([y_labeled,pseudo_labels],dim=0)
        
        #if epoch < swa_start:
            #current_optimizer = optimizer
        #else:
            #current_optimizer = swa_optimizer
        
        optimizer.zero_grad()
        #output_x1 = model(data)
        #output_x1.detach_()
        #optimizer.zero_grad()
        
        images, targets_a, targets_b, lam = mixup_data(data, target, alpha, device)
        
        # calculate robust loss
        loss_trades = trades_loss(model=model,
                           x_natural=images,
                           y=target,
                           optimizer=optimizer,
                           step_size=0.007,
                           epsilon=0.031,
                           perturb_steps=10,
                           beta=6.0)
        
        outputs = model(images)
        #prob = F.softmax(output_x1, dim=1)
        prob_mixup, loss_reg = loss_mixup_reg_ep(outputs, target, targets_a, targets_b, device, lam)
        
        loss=loss_trades+loss_reg
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #loss_ema = loss_ema * 0.9 + float(loss) * 0.1

        # print progress
        if batch_idx % log_interval == 0:
            logging.info('Train Epoch:{}\tLoss: {:.6f}'.format(epoch,loss.item()))
        #logging.info('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                #epoch, batch_idx * len(data), len(train_loader.dataset),
                       #100. * batch_idx / len(train_loader), loss.item()))
    if epoch > swa_start and epoch%swa_freq == 0 :
        swa_optimizer.update_swa()
    if epoch >= swa_start:
        swa_optimizer.bn_update(train_loader, model, device)


def train(model, device, train_loader, optimizer, swa_optimizer, epoch):
    alpha=1
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        
        images, targets_a, targets_b, lam = mixup_data(data, target, alpha, device)

        # calculate robust loss
        loss_trades = trades_loss(model=model,
                           x_natural=data,
                           y=target,
                           optimizer=optimizer,
                           step_size=step_size,
                           epsilon=epsilon,
                           perturb_steps=num_steps,
                           beta=beta)
        
        outputs = model(images)
        prob_mixup, loss_reg = loss_mixup_reg_ep(outputs, target, targets_a, targets_b, device, lam)
        
        loss=loss_trades+loss_reg
        
        loss.backward()
        optimizer.step()
        
        
        # print progress
        if batch_idx % log_interval == 0:
            logging.info('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset), 100. * batch_idx / len(train_loader), loss.item()))
        
    if epoch > swa_start and epoch%swa_freq == 0 :
        swa_optimizer.update_swa()
    if epoch >= swa_start:
        swa_optimizer.bn_update(train_loader, model, device)

        
            
"""
def eval_train(model, device, train_loader):
    model.eval()
    train_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            train_loss += F.cross_entropy(output, target, size_average=False).item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
    train_loss /= len(train_loader.dataset)
    print('Training: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        train_loss, correct, len(train_loader.dataset),
        100. * correct / len(train_loader.dataset)))
    training_accuracy = correct / len(train_loader.dataset)
    return train_loss, training_accuracy
"""
def eval_train(model, device, train_loader):
    model.eval()
    train_loss = 0
    correct = 0
    loss = 0
    total = 0
    correct = 0
    adv_correct = 0
    adv_correct_clean = 0
    adv_total = 0
    print_freq = 25
    
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            train_loss += F.cross_entropy(output, target, reduction='sum').item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            
            if batch_idx < eval_attack_batches:
                is_correct_clean, is_correct_rob = pgd(model, data, target,epsilon=epsilon,num_steps=2 * pgd_num_steps,step_size= pgd_step_size,random_start=False)
                incorrect_clean = (1-is_correct_clean).sum()
                incorrect_rob = (1-np.prod(is_correct_rob, axis=1)).sum()
            
                adv_correct_clean += (len(data) - int(incorrect_clean))
                adv_correct += (len(data) - int(incorrect_rob))
                adv_total += len(data)
            total += len(data)
    train_loss /= len(train_loader.dataset)
    if adv_total > 0:
        robust_clean_accuracy = adv_correct_clean / adv_total
        robust_accuracy = adv_correct / adv_total
    else:
        robust_accuracy = robust_clean_accuracy = 0.
    logging.info('Training: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%),'
        'PDG clean accuracy: {}/{} ({:.2f}%), Robust accuracy {}/{} ({:.2f}%)'.format(
        train_loss, correct, len(train_loader.dataset),
        100. * correct / len(train_loader.dataset),
        adv_correct_clean, adv_total, 100.0 * robust_clean_accuracy,
        adv_correct, adv_total, 100.0 * robust_accuracy))
    training_accuracy = correct / len(train_loader.dataset)
    return train_loss, training_accuracy, robust_accuracy



def eval_test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    correct = 0
    adv_correct = 0
    adv_correct_clean = 0
    adv_total = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            
            if batch_idx < eval_attack_batches:
                is_correct_clean, is_correct_rob = pgd(
                    model, data, target,
                    epsilon=epsilon,
                    num_steps=2 * pgd_num_steps,
                    step_size= pgd_step_size,
                    random_start=False)
                incorrect_clean = (1-is_correct_clean).sum()
                incorrect_rob = (1-np.prod(is_correct_rob, axis=1)).sum()
            
                adv_correct_clean += (len(data) - int(incorrect_clean))
                adv_correct += (len(data) - int(incorrect_rob))
                adv_total += len(data)
            total += len(data)
            
    test_loss /= len(test_loader.dataset)
    if adv_total > 0:
        robust_clean_accuracy = adv_correct_clean / adv_total
        robust_accuracy = adv_correct / adv_total
    else:
        robust_accuracy = robust_clean_accuracy = 0.
    logging.info('Test: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%),'
        'PDG clean accuracy: {}/{} ({:.2f}%), Robust accuracy {}/{} ({:.2f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset),
        adv_correct_clean, adv_total, 100.0 * robust_clean_accuracy,
        adv_correct, adv_total, 100.0 * robust_accuracy))
    test_accuracy = correct / len(test_loader.dataset)
    return test_loss, test_accuracy, robust_accuracy


def adjust_learning_rate(optimizer, epoch):
    """decrease the learning rate"""
    lr = 0.01
    if epoch >= 90:
        lr = lr * 0.1
    if epoch >= 130:
        lr = lr * 0.01
    if epoch >= 140:
        lr = lr * 0.001
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

        
        
def my_collate(batch):
    """Define collate_fn myself because the default_collate_fn throws errors like crazy"""
    # item: a tuple of (img, label)
    data = [item[0] for item in batch]
    target = [item[1] for item in batch]
    data = torch.stack(data)
    target = torch.LongTensor(target)
    return [data, target]

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim


def squared_l2_norm(x):
    flattened = x.view(x.unsqueeze(0).shape[0], -1)
    return (flattened ** 2).sum(1)


def l2_norm(x):
    return squared_l2_norm(x).sqrt()


def trades_loss(model,
                x_natural,
                y,
                optimizer,
                step_size=0.003,
                epsilon=0.031,
                perturb_steps=10,
                beta=6,
                distance='l_inf'):
    # define KL-loss
    criterion_kl = nn.KLDivLoss(reduction='sum')
    model.eval()
    batch_size = len(x_natural)
    # generate adversarial example
    x_adv = x_natural.detach() + 0.001 * torch.randn(x_natural.shape).cuda().detach()
    if distance == 'l_inf':
        for _ in range(perturb_steps):
            x_adv.requires_grad_()
            with torch.enable_grad():
                loss_kl = criterion_kl(F.log_softmax(model(x_adv), dim=1),
                                       F.softmax(model(x_natural), dim=1))
            grad = torch.autograd.grad(loss_kl, [x_adv])[0]
            x_adv = x_adv.detach() + step_size * torch.sign(grad.detach())
            x_adv = torch.min(torch.max(x_adv, x_natural - epsilon), x_natural + epsilon)
            x_adv = torch.clamp(x_adv, 0.0, 1.0)
    elif distance == 'l_2':
        delta = 0.001 * torch.randn(x_natural.shape).cuda().detach()
        delta = Variable(delta.data, requires_grad=True)

        # Setup optimizers
        optimizer_delta = optim.SGD([delta], lr=epsilon / perturb_steps * 2)

        for _ in range(perturb_steps):
            adv = x_natural + delta

            # optimize
            optimizer_delta.zero_grad()
            with torch.enable_grad():
                loss = (-1) * criterion_kl(F.log_softmax(model(adv), dim=1),
                                           F.softmax(model(x_natural), dim=1))
            loss.backward()
            # renorming gradient
            grad_norms = delta.grad.view(batch_size, -1).norm(p=2, dim=1)
            delta.grad.div_(grad_norms.view(-1, 1, 1, 1))
            # avoid nan or inf if gradient is 0
            if (grad_norms == 0).any():
                delta.grad[grad_norms == 0] = torch.randn_like(delta.grad[grad_norms == 0])
            optimizer_delta.step()

            # projection
            delta.data.add_(x_natural)
            delta.data.clamp_(0, 1).sub_(x_natural)
            delta.data.renorm_(p=2, dim=0, maxnorm=epsilon)
        x_adv = Variable(x_natural + delta, requires_grad=False)
    else:
        x_adv = torch.clamp(x_adv, 0.0, 1.0)
    model.train()

    x_adv = Variable(torch.clamp(x_adv, 0.0, 1.0), requires_grad=False)
    # zero gradient
    optimizer.zero_grad()
    # calculate robust loss
    logits = model(x_natural)
    loss_natural = F.cross_entropy(logits, y)
    loss_robust = (1.0 / batch_size) * criterion_kl(F.log_softmax(model(x_adv), dim=1),
                                                    F.softmax(model(x_natural), dim=1))
    loss = loss_natural + beta * loss_robust
    return loss

In [15]:
def main():
    # init model, ResNet18() can be also used here for training
    #os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3"
    #device_ids = [1, 2, 3]
    #device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    num_classes = 10
    swa_lr = 0.0001
    model = CNN(num_classes=10, isL2 = False, dropRatio = 0.1)
    #model = get_model('wrn-28-10', num_classes=num_classes,
    #                      normalize_input=False)
    #model = model.to(device)

    if torch.cuda.device_count() > 1:
        model = DataParallel(model)
        print("Using", torch.cuda.device_count(), "GPUs!")
        model = model.to(device)
    lr =0.01
    print_freq = 50
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
    #optimizer = SWA(base_optimizer, swa_lr=swa_lr)
    swa_optimizer = SWA(optimizer, swa_start=swa_start, swa_freq=swa_freq, swa_lr=swa_lr)
    criterion = nn.CrossEntropyLoss()
    save_freq = 50
    sup_epochs = 150
    
    
    model.train()
    optimizer.zero_grad()
    
    # Forward pass for labeled data
    writer = SummaryWriter("runs/mixup/fly_soft_cnn_swa70_lowBS_50_50_50k_adversarial")
    for epoch in tqdm(range(sup_epochs)):
        lr = adjust_learning_rate(optimizer, epoch)
        #train(model, device, labeled_loader, optimizer, swa_optimizer, epoch)
        train(model, device, train_loader, optimizer, swa_optimizer, epoch)
        
        print('================================================================')
        cnn_swa_test_loss, cnn_swa_test_accuracy, cnn_swa_test_robust_accuracy = eval_test(model, device, test_loader)
        writer.add_scalar('fly_soft_swa70_robust_acc/test', cnn_swa_test_robust_accuracy, epoch)
        writer.add_scalar('fly_soft_swa70_clean_acc/test', cnn_swa_test_accuracy, epoch)
        print('================================================================')
        
        
        # save checkpoint
        if epoch % save_freq == 0:
            torch.save(model.state_dict(),
                       os.path.join(model_dir, 'fly_soft_swa70_lowBS_50_50_50k_model-cnn-epoch{}.pt'.format(epoch)))
            torch.save(optimizer.state_dict(),
                       os.path.join(model_dir, 'fly_soft_swa70_lowBS_50_50_50k_opt-cnn-checkpoint_epoch{}.tar'.format(epoch)))
    
    swa_optimizer.swap_swa_sgd()
    best_swa_test_loss, best_swa_test_accuracy, best_swa_test_robust_accuracy = eval_test(model, device, test_loader)
        #for x_labeled, y_labeled in labeled_loader:
        #    x_labeled, y_labeled = x_labeled.to(device), y_labeled.to(device)
        #    labeled_output = model(x_labeled)
        #    labeled_loss = criterion(labeled_output, y_labeled)

            # Backward pass and optimization
        #    labeled_loss.backward()
        #    optimizer.step()
        #    optimizer.zero_grad()

    #writer = SummaryWriter("runs/mixup/fly_soft_cnn_swa70_lowBS_50_50_1k_adversarial")
    #for epoch in tqdm(range(1, epochs + 1)):
        #lr = adjust_learning_rate(optimizer, epoch)
        #train_new(model, device, labeled_loader, unlabeled_loader, optimizer, swa_optimizer, epoch)
        #train(model, device, train_loader, optimizer, swa_optimizer, epoch)
        
        

        # evaluation on natural examples
        #print('================================================================')
        #val_train_loss, val_train_accuracy, val_robust_accuracy = eval_train(model, device, labeled_validation_loader)
        #writer.add_scalar('robust_acc/val', val_robust_accuracy, epoch)
        #writer.add_scalar('clean_acc/val', val_train_accuracy, epoch)
        
        #cnn_swa_test_loss, cnn_swa_test_accuracy, cnn_swa_test_robust_accuracy = eval_test(model, device, test_loader)
        #writer.add_scalar('fly_soft_swa70_robust_acc/test', cnn_swa_test_robust_accuracy, epoch)
       # writer.add_scalar('fly_soft_swa70_clean_acc/test', cnn_swa_test_accuracy, epoch)
        #print('================================================================')

        
       
        # save checkpoint
        #if epoch % save_freq == 0:
        #    torch.save(model.state_dict(),
        #               os.path.join(model_dir, 'fly_soft_swa70_lowBS_50_50_1k_model-cnn-epoch{}.pt'.format(epoch)))
        #    torch.save(optimizer.state_dict(),
        #               os.path.join(model_dir, 'fly_soft_swa70_lowBS_50_50_1k_opt-cnn-checkpoint_epoch{}.tar'.format(epoch)))
    
    #swa_optimizer.swap_swa_sgd()
    #best_swa_test_loss, best_swa_test_accuracy, best_swa_test_robust_accuracy = eval_test(model, device, test_loader)


if __name__ == '__main__':
    main()

Using 4 GPUs!


  0%|          | 0/150 [00:00<?, ?it/s]

2023-10-23 13:16:38,200 | Train Epoch: 0 [0/50000 (0%)]	Loss: 7.490875
2023-10-23 13:17:32,375 | Train Epoch: 0 [5000/50000 (10%)]	Loss: 6.968800
2023-10-23 13:18:24,781 | Train Epoch: 0 [10000/50000 (20%)]	Loss: 6.572667
2023-10-23 13:19:18,119 | Train Epoch: 0 [15000/50000 (30%)]	Loss: 6.183801
2023-10-23 13:20:11,511 | Train Epoch: 0 [20000/50000 (40%)]	Loss: 5.898573
2023-10-23 13:21:03,844 | Train Epoch: 0 [25000/50000 (50%)]	Loss: 5.615528
2023-10-23 13:21:57,295 | Train Epoch: 0 [30000/50000 (60%)]	Loss: 6.399069
2023-10-23 13:22:50,037 | Train Epoch: 0 [35000/50000 (70%)]	Loss: 6.042807
2023-10-23 13:23:42,565 | Train Epoch: 0 [40000/50000 (80%)]	Loss: 6.387190
2023-10-23 13:24:34,643 | Train Epoch: 0 [45000/50000 (90%)]	Loss: 6.129626


2023-10-23 13:25:30,205 | Test: Average loss: 1.2049, Accuracy: 5673/10000 (57%),PDG clean accuracy: 70/128 (54.69%), Robust accuracy 29/128 (22.66%)


2023-10-23 13:25:30,970 | Train Epoch: 1 [0/50000 (0%)]	Loss: 5.764554
2023-10-23 13:26:23,933 | Train Epoch: 1 [5000/50000 (10%)]	Loss: 6.337267
2023-10-23 13:27:16,803 | Train Epoch: 1 [10000/50000 (20%)]	Loss: 5.667315
2023-10-23 13:28:09,464 | Train Epoch: 1 [15000/50000 (30%)]	Loss: 5.993816
2023-10-23 13:29:03,213 | Train Epoch: 1 [20000/50000 (40%)]	Loss: 5.252282
2023-10-23 13:29:55,933 | Train Epoch: 1 [25000/50000 (50%)]	Loss: 6.298725
2023-10-23 13:30:48,030 | Train Epoch: 1 [30000/50000 (60%)]	Loss: 5.292431
2023-10-23 13:31:41,699 | Train Epoch: 1 [35000/50000 (70%)]	Loss: 5.850338
2023-10-23 13:32:34,047 | Train Epoch: 1 [40000/50000 (80%)]	Loss: 5.502298
2023-10-23 13:33:27,577 | Train Epoch: 1 [45000/50000 (90%)]	Loss: 5.272674


...

2023-10-24 11:45:02,706 | Train Epoch: 148 [0/50000 (0%)]	Loss: 4.213702
2023-10-24 11:45:56,029 | Train Epoch: 148 [5000/50000 (10%)]	Loss: 4.080365
2023-10-24 11:46:48,923 | Train Epoch: 148 [10000/50000 (20%)]	Loss: 4.072923
2023-10-24 11:47:41,410 | Train Epoch: 148 [15000/50000 (30%)]	Loss: 2.711586
2023-10-24 11:48:34,402 | Train Epoch: 148 [20000/50000 (40%)]	Loss: 2.631715
2023-10-24 11:49:27,502 | Train Epoch: 148 [25000/50000 (50%)]	Loss: 2.907120
2023-10-24 11:50:20,552 | Train Epoch: 148 [30000/50000 (60%)]	Loss: 2.984087
2023-10-24 11:51:13,287 | Train Epoch: 148 [35000/50000 (70%)]	Loss: 3.343543
2023-10-24 11:52:05,929 | Train Epoch: 148 [40000/50000 (80%)]	Loss: 4.602960
2023-10-24 11:52:59,002 | Train Epoch: 148 [45000/50000 (90%)]	Loss: 4.032154


2023-10-24 11:54:16,570 | Test: Average loss: 0.2917, Accuracy: 9034/10000 (90%),PDG clean accuracy: 118/128 (92.19%), Robust accuracy 81/128 (63.28%)


2023-10-24 11:54:17,249 | Train Epoch: 149 [0/50000 (0%)]	Loss: 2.683794
2023-10-24 11:55:09,910 | Train Epoch: 149 [5000/50000 (10%)]	Loss: 3.926421
2023-10-24 11:56:02,740 | Train Epoch: 149 [10000/50000 (20%)]	Loss: 3.957108
2023-10-24 11:56:55,421 | Train Epoch: 149 [15000/50000 (30%)]	Loss: 3.020185
2023-10-24 11:57:47,992 | Train Epoch: 149 [20000/50000 (40%)]	Loss: 4.123168
2023-10-24 11:58:41,742 | Train Epoch: 149 [25000/50000 (50%)]	Loss: 2.696151
2023-10-24 11:59:35,326 | Train Epoch: 149 [30000/50000 (60%)]	Loss: 3.318903
2023-10-24 12:00:27,775 | Train Epoch: 149 [35000/50000 (70%)]	Loss: 3.234161
2023-10-24 12:01:21,169 | Train Epoch: 149 [40000/50000 (80%)]	Loss: 4.227908
2023-10-24 12:02:14,712 | Train Epoch: 149 [45000/50000 (90%)]	Loss: 3.428628


2023-10-24 12:03:31,337 | Test: Average loss: 0.2918, Accuracy: 9033/10000 (90%),PDG clean accuracy: 118/128 (92.19%), Robust accuracy 81/128 (63.28%)


2023-10-24 12:03:34,322 | Test: Average loss: 72.3867, Accuracy: 2656/10000 (27%),PDG clean accuracy: 32/128 (25.00%), Robust accuracy 61/128 (47.66%)


In [ ]:
 from torchcontrib.optim import SWA
        base_optimizer = optim.SGD(model.parameters(), lr=lr, momentum=args.momentum, weight_decay=args.wd)
        optimizer = SWA(base_optimizer, swa_lr=args.swa_lr)

In [ ]:
 # applying swa
    if args.swa == 'True':
        optimizer.swap_swa_sgd()
        optimizer.bn_update(train_loader, model, device)
        if args.validation_exp == "True":
            loss_swa, acc_val_swa = validating(args, model, device, test_loader)

In [ ]:
if args.swa == 'True':
        if epoch > args.swa_start and epoch%args.swa_freq == 0 :
            optimizer.update_swa()

def main():
    # init model, ResNet18() can be also used here for training
    #os.environ["CUDA_VISIBLE_DEVICES"] = "1"
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    num_classes = 10
    model = get_model('wrn-28-10', num_classes=num_classes,
                          normalize_input=False)
    model = model.to(device)

    if torch.cuda.device_count() > 1:
        print("Using", torch.cuda.device_count(), "GPUs!")
        model = DataParallel(model)
    #model = model.to(device)
    lr =0.1
    print_freq = 50
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
    criterion = nn.CrossEntropyLoss()
    save_freq = 50
    
    
    model.train()
    optimizer.zero_grad()
    
    # Forward pass for labeled data
    for x_labeled, y_labeled in labeled_loader:
        x_labeled, y_labeled = x_labeled.to(device), y_labeled.to(device)
        labeled_output = model(x_labeled)
        labeled_loss = criterion(labeled_output, y_labeled)

        # Backward pass and optimization
        labeled_loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    writer = SummaryWriter("runs/4k_adversarial")
    for epoch in tqdm(range(1, epochs + 1)):
        lr = adjust_learning_rate(optimizer, epoch)
        train_new(model, device, labeled_loader, unlabeled_loader, optimizer, epoch)

        # evaluation on natural examples
        print('================================================================')
        val_train_loss, val_train_accuracy, val_robust_accuracy = eval_train(model, device, labeled_validation_loader)
        writer.add_scalar('robust_acc/val', val_robust_accuracy, epoch)
        writer.add_scalar('clean_acc/val', val_train_accuracy, epoch)
        
        test_loss, test_accuracy, test_robust_accuracy = eval_test(model, device, test_loader)
        writer.add_scalar('robust_acc/test', test_robust_accuracy, epoch)
        writer.add_scalar('clean_acc/test', test_accuracy, epoch)
        print('================================================================')

        
       
        # save checkpoint
        if epoch % save_freq == 0:
            torch.save(model.state_dict(),
                       os.path.join(model_dir, '1_4k_model-wideres-epoch{}.pt'.format(epoch)))
            torch.save(optimizer.state_dict(),
                       os.path.join(model_dir, '1_4k_opt-wideres-checkpoint_epoch{}.tar'.format(epoch)))


if __name__ == '__main__':
    main()

In [ ]:
model.train()
optimizer.zero_grad()

writer = SummaryWriter("runs/10k_adversarial")

for epoch in tqdm(range(1, epochs + 1)):
    # adjust learning rate for SGD
    adjust_learning_rate(optimizer, epoch, lr)
    
    # Forward pass for labeled data
    for x_labeled, y_labeled in labeled_loader:
        x_labeled, y_labeled = x_labeled.to(device), y_labeled.to(device)
        labeled_output = model(x_labeled)
        labeled_loss = criterion(labeled_output, y_labeled)

        # Backward pass and optimization
        labeled_loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    # Generate pseudolabels for unlabeled data batch by batch
    model.eval()
    for x_unlabeled, _ in unlabeled_loader:
        data_array = x_unlabeled.to(device)
        unlabeled_output = model(data_array)
        pseudo_labels_batch = unlabeled_output.argmax(dim=1)

        # Convert data_array and pseudo_labels_batch to tensors
        data_array = data_array.cpu().numpy()
        pseudo_labels_batch = pseudo_labels_batch.cpu().numpy()

        # Perform training with labeled and pseudo-labeled batch
        combined_dataset = torch.utils.data.TensorDataset(torch.from_numpy(data_array),
                                                         torch.from_numpy(pseudo_labels_batch))
        combined_loader = torch.utils.data.DataLoader(combined_dataset,
                                                      batch_size=TRAIN_BS,
                                                      shuffle=True,
                                                      num_workers=2,
                                                      collate_fn=my_collate)

        train(model, device, combined_loader, optimizer, epoch)

    # evaluation on natural examples
    print('================================================================')
    train_loss, training_accuracy = eval_train(model, device, labeled_loader)
    eval_test(model, device, test_loader)
    print('================================================================')


In [ ]:
def main():
    # init model, ResNet18() can be also used here for training
    #os.environ["CUDA_VISIBLE_DEVICES"] = "1"
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    num_classes = 10
    model = get_model('wrn-28-10', num_classes=num_classes,
                          normalize_input=False)
    model = model.to(device)

    if torch.cuda.device_count() > 1:
        print("Using", torch.cuda.device_count(), "GPUs!")
        model = DataParallel(model)
    #model = model.to(device)
    lr =0.1
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
    criterion = nn.CrossEntropyLoss()
    save_freq = 50
    # Load the combined_loader
    #with open('./data/combined_loader.pkl', 'rb') as f:
    #    combined_loader = pickle.load(f)
    
    model.train()
    optimizer.zero_grad()

    # Forward pass
    for x_labeled, y_labeled in labeled_loader:
        x_labeled, y_labeled = x_labeled.to(device), y_labeled.to(device)
        labeled_output = model(x_labeled)
        labeled_loss = criterion(labeled_output, y_labeled)

        # Backward pass and optimization
        labeled_loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    
    
    for epoch in tqdm(range(1, epochs + 1)):
        # adjust learning rate for SGD
        adjust_learning_rate(optimizer, epoch, lr)
        

        # Generate pseudo-labels for unlabeled data
        model.eval()
        pseudo_labels = []
        data_array = []
        with torch.no_grad():
            for x_unlabeled, _ in unlabeled_loader:
                data_array.append(x_unlabeled.numpy())
                x_unlabeled = x_unlabeled.to(device)
                
                unlabeled_output = model(x_unlabeled)
                pseudo_labels_batch = unlabeled_output.argmax(dim=1)
                pseudo_labels.append(pseudo_labels_batch.cpu().numpy())
        
        
        data_array = np.concatenate(data_array)
        new_extrapolated_targets = np.concatenate(pseudo_labels)
        
        # Convert data_array and new_extrapolated_targets to tensors
        data_array = torch.from_numpy(data_array)
        new_extrapolated_targets = torch.from_numpy(new_extrapolated_targets)
        
        
        # Create a new dataset by combining labeled data and predicted labels
        pseudolabel_combined_dataset = torch.utils.data.TensorDataset(data_array, new_extrapolated_targets)

        # Create a combined dataloader
        pseudolabel_combined_loader = torch.utils.data.DataLoader(pseudolabel_combined_dataset, batch_size=TRAIN_BS, shuffle=True, num_workers=2)
        # Combine labeled and pseudo-labeled data
        # Create a new pseudo-labels dataset
        combined_dataset = torch.utils.data.ConcatDataset([labeled_dataset, pseudolabel_combined_dataset])
        combined_loader = torch.utils.data.DataLoader(
            combined_dataset, batch_size=TRAIN_BS, shuffle=True, num_workers=2, collate_fn=my_collate)

        
 
        
        
        # adversarial training
        train(model, device, combined_loader, optimizer, epoch)

        # evaluation on natural examples
        print('================================================================')
        train_loss, training_accuracy = eval_train(model, device, labeled_loader)
        eval_test(model, device, test_loader)
        print('================================================================')

        
        writer.add_scalar('robust_accuracy', np.array(training_accuracy), epoch)
        # save checkpoint
        if epoch % save_freq == 0:
            torch.save(model.state_dict(),
                       os.path.join(model_dir, '10k_model-wideres-epoch{}.pt'.format(epoch)))
            torch.save(optimizer.state_dict(),
                       os.path.join(model_dir, '10k_opt-wideres-checkpoint_epoch{}.tar'.format(epoch)))


if __name__ == '__main__':
    main()





In [ ]:
len(unlabeled_dataset)

In [ ]:
len(c_dataset)

In [ ]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()

        # calculate robust loss
        loss = trades_loss(model=model,
                           x_natural=data,
                           y=target,
                           optimizer=optimizer,
                           step_size=step_size,
                           epsilon=epsilon,
                           perturb_steps=num_steps,
                           beta=beta)
        loss.backward()
        optimizer.step()

        # print progress
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))

In [ ]:
class SemiSupervisedSampler(Sampler):
    """Balanced sampling from the labeled and unlabeled data"""
    def __init__(self, sup_inds, unsup_inds, batch_size, unsup_fraction=0.5,
                 num_batches=None):
        if unsup_fraction is None or unsup_fraction < 0:
            self.sup_inds = sup_inds + unsup_inds
            unsup_fraction = 0.0
        else:
            self.sup_inds = sup_inds
            self.unsup_inds = unsup_inds

        self.batch_size = batch_size
        unsup_batch_size = int(batch_size * unsup_fraction)
        self.sup_batch_size = batch_size - unsup_batch_size

        if num_batches is not None:
            self.num_batches = num_batches
        else:
            self.num_batches = int(
                np.ceil(len(self.sup_inds) / self.sup_batch_size))

        super().__init__(None)

    def __iter__(self):
        batch_counter = 0
        while batch_counter < self.num_batches:
            sup_inds_shuffled = [self.sup_inds[i]
                                 for i in torch.randperm(len(self.sup_inds))]
            for sup_k in range(0, len(self.sup_inds), self.sup_batch_size):
                if batch_counter == self.num_batches:
                    break
                batch = sup_inds_shuffled[sup_k:(sup_k + self.sup_batch_size)]
                if self.sup_batch_size < self.batch_size:
                    batch.extend([self.unsup_inds[i] for i in
                                  torch.randint(high=len(self.unsup_inds),
                                                size=(
                                                    self.batch_size - len(
                                                        batch),),
                                                dtype=torch.int64)])
                # this shuffle operation is very important, without it
                # batch-norm / DataParallel hell ensues
                np.random.shuffle(batch)
                yield batch
                batch_counter += 1

    def __len__(self):
        return self.num_batches


In [ ]:
sup_inds = [1, 2, 3, 4, 5]  # Labeled data indices
unsup_inds = [6, 7, 8, 9, 10]  # Unlabeled data indices
batch_size = 4
unsup_fraction = 0.5

sampler = SemiSupervisedSampler(sup_inds, unsup_inds, batch_size, unsup_fraction)
dataloader = torch.utils.data.DataLoader(dataset, sampler=sampler, batch_size=batch_size)

for batch_indices in dataloader:
    # Use batch_indices for training
    print(batch_indices)

In [ ]:
for batch_data, batch_labels in combined_loader:
    print("Batch data shape:", batch_data.shape)
    print("Batch labels:", batch_labels)


In [ ]:
def train_new(model, device, labeled_loader, unlabeled_loader, optimizer, swa_optimizer, epoch):    

    labeled_iter = iter(labeled_loader)    
    num_iter = (len(unlabeled_loader.dataset)//TRAIN_BS)+1
    for batch_idx, (x_unlabeled, _) in enumerate(unlabeled_loader):      
        try:
            x_labeled, y_labeled = next(labeled_iter)
        except StopIteration:
            labeled_iter = iter(labeled_loader)
            x_labeled, y_labeled = next(labeled_iter) 
        model.eval()
        with torch.no_grad():
            x_unlabeled = x_unlabeled.to(device)
            
            logits_unlabeled = model(x_unlabeled.to(device))
            # Softmax normalization to obtain probabilities for each class
            pseudo_labels = F.softmax(logits_unlabeled, dim=1)
            
            
            
        
        model.train()
        
        # concatenate (x_labeled, y_labeled) and (x_unlabeled, pseudo_labels)
        x_labeled = x_labeled.to(device)
        y_labeled = y_labeled.to(device)
        #print(y_labeled.shape)
        #print(y_labeled[0])
        # Add a new dimension to y_labeled to make it compatible for concatenation
        num_classes = 10
        y_labeled_onehot = F.one_hot(y_labeled, num_classes)
        #print(y_labeled_onehot.shape)
        #print(y_labeled_onehot[0])
        pseudo_labels = pseudo_labels.to(device)
        #print(pseudo_labels.shape)
        
        data = torch.cat([x_labeled,x_unlabeled],dim=0)
        target = torch.cat([y_labeled,pseudo_labels],dim=0)
        
        optimizer.zero_grad()
        # calculate robust loss
        loss = trades_loss(model=model,
                           x_natural=data,
                           y=target,
                           optimizer=optimizer,
                           step_size=0.007,
                           epsilon=0.031,
                           perturb_steps=10,
                           beta=beta)
        loss.backward()
        optimizer.step()


In [ ]:
""""
for images,labels in combined_loader:
         print(type(images))
         print(type(labels))
         print(images.size())
         print(images.dim())
""""

In [ ]:
""""
model_dir= 'rst_adv/pseudolabel_cifar10/10k'
epoch = 100
normalize_input = False
checkpoint = torch.load(os.path.join(model_dir, 'checkpoint-epoch%d.pt' % epoch))
num_classes = checkpoint.get('num_classes', 10)
normalize_input = checkpoint.get('normalize_input', False)
model = get_model('wrn-28-10', 
                  num_classes=num_classes,
                  normalize_input=normalize_input)
model = model.to(device)
#model = nn.DataParallel(model).cuda()
model.load_state_dict(checkpoint['state_dict'])
"""

In [ ]:
"""predictions = []
data_array = []
for i, (batch, _) in enumerate(unlabeled_loader):
    data_array.append(batch.numpy())
    _, preds = torch.max(model(batch.to(device)), dim=1)
    predictions.append(preds.cpu().numpy())
    
    if (i+1) % 10 == 0:
        print('Done %d/%d' % (i+1, len(unlabeled_loader)))
data_array = np.concatenate(data_array)
new_extrapolated_targets = np.concatenate(predictions)

# Convert data_array and new_extrapolated_targets to tensors
data_array = torch.from_numpy(data_array)
new_extrapolated_targets = torch.from_numpy(new_extrapolated_targets)

# Create a new dataset by combining labeled data and predicted labels
pseudolabel_combined_dataset = torch.utils.data.TensorDataset(data_array, new_extrapolated_targets)

# Create a combined dataloader
pseudolabel_combined_loader = torch.utils.data.DataLoader(pseudolabel_combined_dataset, batch_size=TRAIN_BS, shuffle=True, num_workers=2)

def my_collate(batch):
    """Define collate_fn myself because the default_collate_fn throws errors like crazy"""
    # item: a tuple of (img, label)
    data = [item[0] for item in batch]
    target = [item[1] for item in batch]
    data = torch.stack(data)
    target = torch.LongTensor(target)
    return [data, target]

combined_dataset = torch.utils.data.ConcatDataset([labeled_dataset, pseudolabel_combined_dataset])
combined_loader = torch.utils.data.DataLoader(
    combined_dataset, batch_size=TRAIN_BS, shuffle=True, num_workers=2, collate_fn=my_collate)


# Save the combined_loader
with open('./data/combined_loader.pkl', 'wb') as f:
    pickle.dump(combined_loader, f)
    """

In [ ]:
import torch
import torch.nn.functional as F

# Example data
x_labeled = torch.randn((5, 3, 32, 32))  # Example labeled data
y_labeled = torch.tensor([1, 0, 2, 1, 0])  # Example class indices for labeled data

x_unlabeled = torch.randn((5, 3, 32, 32))  # Example unlabeled data

# Example model
class YourModel(torch.nn.Module):
    def __init__(self):
        super(YourModel, self).__init__()
        self.fc = torch.nn.Linear(3 * 32 * 32, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        return self.fc(x)

model = YourModel()

# Set the model to evaluation mode for pseudo-label generation
model.eval()

# Generate pseudo-labels
with torch.no_grad():
    logits_unlabeled = model(x_unlabeled)
    pseudo_labels = F.softmax(logits_unlabeled, dim=1)

# Ensure y_labeled has the same number of classes as the model
num_classes = 10
y_labeled = y_labeled.view(-1, 1).long()
y_labeled_onehot = F.one_hot(y_labeled, num_classes).float()  # Convert to one-hot encoding

# Concatenate class indices and softmax probabilities
target = torch.cat([y_labeled_onehot, pseudo_labels], dim=0)

print("Pseudo Labels:")
print(pseudo_labels)
print("Target (Concatenated):")
print(target)


In [ ]:
import torch
import torch.nn.functional as F

# Example hard labels (class indices)
hard_labels = torch.tensor([1, 0, 2, 1, 0])  # Example shape: (batch_size,)

# Example number of classes
num_classes = 10

# Convert hard labels to one-hot encoding
hard_labels_onehot = F.one_hot(hard_labels, num_classes).float()

# Example soft label probabilities
soft_labels = F.softmax(torch.randn(5, 10), dim=1)  # Example shape: (batch_size, num_classes)

# Concatenate hard labels in one-hot encoding and soft labels
concatenated_labels = torch.cat([hard_labels_onehot, soft_labels], dim=0)

print("Hard Labels (One-Hot):")
print(hard_labels_onehot)
print("Soft Labels:")
print(soft_labels)
print("Concatenated Labels:")
print(concatenated_labels)
